In [6]:
# Mount your google drive. 
# Use this to save your PyTorch model for submission
from google.colab import drive
drive.mount('/content/gdrive')
!mkdir /content/gdrive/Team\ Drives/cis530
#Test drive access. 
#You should have a test.txt in your Google drive
with open('/content/gdrive/Team Drives/cis530/test.txt', 'w') as f:
  f.write('This is a test file!')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
mkdir: cannot create directory ‘/content/gdrive/Team Drives/cis530’: File exists


In [1]:
from nltk.corpus import conll2002
from nltk.corpus import cess_esp as cess

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Perceptron
from sklearn.metrics import precision_recall_fscore_support

from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.svm import LinearSVC
import pickle
!pip3 install wordfreq
from collections import OrderedDict 
from wordfreq import zipf_frequency

In [0]:
def isApostrophePresent(word):
    if "'" in word:
        return True
    return False

  
def isDashPresent(word):
    if "-" in word:
        return True
    return False

def oneDigit(word):
    num = 0
    for l in word:
      if l.isdigit():
        num +=1
    if num == 1:
      return True
    else:
      return False

def twoDigits(word):
    num = 0
    for l in word:
      if l.isdigit():
        num +=1
    if num == 2:
      return True
    else:
      return False
    
def threeDigits(word):
    num = 0
    for l in word:
      if l.isdigit():
        num +=1
    if num == 3:
      return True
    else:
      return False
    
def fiveDigits(word):
    num = 0
    for l in word:
      if l.isdigit():
        num +=1
    if num == 5:
      return True
    else:
      return False
    
def sixDigits(word):
    num = 0
    for l in word:
      if l.isdigit():
        num +=1
    if num == 6:
      return True
    else:
      return False
    
def sevenDigits(word):
    num = 0
    for l in word:
      if l.isdigit():
        num +=1
    if num == 7:
      return True
    else:
      return False
    
def nineDigits(word):
    num = 0
    for l in word:
      if l.isdigit():
        num +=1
    if num == 9:
      return True
    else:
      return False

def fax(word):
    if "fax" in word.lower():
      return True
    else:
      return False

In [0]:
def getfeats(word, postag, o):
    """ This takes the word in question and
    the offset with respect to the instance
    word """
    o = str(o)
    features = [
        (o + 'word', word),
        # TODO: add more features here.
                
        (o + 'word', word),
        # TODO: add more features here.
        (o + 'word.isupper', any(letter.isupper() for letter in word)), #4
        (o + 'word.len', len(word) ), #2
        (o + 'isApostrophePresent', isApostrophePresent(word)),
        (o + 'isDashPresent', isDashPresent(word)),
        
        (o + 'oneDigit', oneDigit(word)),
        (o + 'twoDigits', twoDigits(word)),
        (o + 'threeDigits', threeDigits(word)),
        (o + 'fiveDigits', fiveDigits(word)),
        (o + 'sixDigits', sixDigits(word)),
        (o + 'sevenDigits', sevenDigits(word)),
        (o + 'nineDigits', nineDigits(word)),
        
        (o + 'fax', fax(word)),
        (o + 'isupper', word.isupper()), #13
        (o + 'islower', word.islower()), #14
        (o + 'word.wordfreq', zipf_frequency(word, 'es') ) #15
        
        
        

#         (o + 'word[-3:]', word[-3:]),
#         (o + 'word[-2:]', word[-2:]),
#         (o + 'word.istitle()', word.istitle()),
#         (o + 'word.isdigit()', word.isdigit()),
#         (o + 'postag', postag),
#         (o + 'postag[:2]', postag[:2]),
    ]
    return features

In [0]:
def word2features(sent, i):
    """ The function generates all features
    for the word at position i in the
    sentence."""
    features = []
    # the window around the token
    featlist = [('bias', 1.0)]
    features.extend(featlist)

    for o in [-1,0,1,2]:
        if i+o >= 0 and i+o < len(sent):
            word = sent[i+o][0]
#             postag = sent[i+o][1]
            postag = "unknown"
            featlist = getfeats(word, postag, o)
            features.extend(featlist)
        elif i+o<0:
            featlist = [('BOS', 1)]
            features.extend(featlist)
        else:
            featlist = [('EOS', 1)]
            features.extend(featlist)    
    return dict(features)

In [14]:

if __name__ == "__main__":
  
    # Load the training data
    file = open('/content/gdrive/Team Drives/cis530/preprocessed_data/word_ner_startindex_dict/train_word_ner_startidx_dict.pickle','rb')
    train_dict = pickle.load(file)
    
    #ADDED RIGHT NOW!
    train_dict = OrderedDict(train_dict)
    
    file.close()
    
    train_sents = []
    
    for k, v in train_dict.items():
      train_sents.extend(v)

    print("train_sents len= ", len(train_sents))

    file = open('/content/gdrive/Team Drives/cis530/preprocessed_data/word_ner_startindex_dict/dev_word_ner_startidx_dict.pickle','rb')
    dev_dict = pickle.load(file)
    
    #ADDED RIGHT NOW!
    dev_dict = OrderedDict(dev_dict)
    file.close()
    
    dev_sents = []
    for k, v in dev_dict.items():
      dev_sents.extend(v)
    
    print("dev_sents len= ", len(dev_sents))
      
    file = open('/content/gdrive/Team Drives/cis530/preprocessed_data/word_ner_startindex_dict/test_word_ner_startidx_dict.pickle','rb')
    test_dict = pickle.load(file)
    
    #ADDED RIGHT NOW!
    test_dict = OrderedDict(test_dict)
    file.close()
    
    test_sents = []
    for k, v in test_dict.items():
      test_sents.extend(v)
    
    print("test_sents len= ", len(test_sents))
    
    train_feats = []
    train_labels = []

    for sent in train_sents:
        for i in range(len(sent)):
            feats = word2features(sent,i)
            train_feats.append(feats)
            train_labels.append(sent[i][1])

    vectorizer = DictVectorizer()
    X_train = vectorizer.fit_transform(train_feats)

#     # TODO: play with other models
#     # model = Perceptron(verbose=1)
#     # model = MultinomialNB(alpha=0.01)
#     #model = PassiveAggressiveClassifier(C=1.0, fit_intercept=True, early_stopping=False, loss='hinge', average=True, random_state=99)
#     # model = SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, fit_intercept=True, random_state=99)

    print('Training the model')
    model = LinearSVC()
    
    model.fit(X_train, train_labels)
    print ('Trained the model')

# ##     pickle.dump(model, open('model', 'wb'))

    #Training Data

    y_train_pred = model.predict(X_train)

#     j = 0
#     print("Writing to train_results.txt")
#     # format is: word gold pred
#     with open("/content/gdrive/Team Drives/cis530/preprocessed_data/word_ner_startindex_dict/train_results.txt", "w") as out:
#         for sent in train_sents: 
#             for i in range(len(sent)):
#                 word = sent[i][0]
#                 gold = sent[i][1]
#                 pred = y_train_pred[j]
#                 j += 1
#                 out.write("{}\t{}\t{}\n".format(word,gold,pred))
#         out.write("\n")

    train_pathRead = "/content/gdrive/Team Drives/cis530/raw_system_data/train/system/"
    train_pathOut = "/content/gdrive/Team Drives/cis530/raw_system_data/train/system/"
#     #create an .ann format for predictions
    createAnnFormat(train_dict, y_train_pred,train_pathRead, train_pathOut)
    

         
    #Dev Data
    dev_feats = []
    dev_labels = []

    # switch to test_sents for your final results
    for sent in dev_sents:
        for i in range(len(sent)):
            feats = word2features(sent,i)
            dev_feats.append(feats)
            dev_labels.append(sent[i][1])

    X_dev = vectorizer.transform(dev_feats)
    y_dev_pred = model.predict(X_dev)

#     j = 0
#     print("Writing to dev_results.txt")
#     # format is: word gold pred
#     with open("/content/gdrive/Team Drives/cis530/preprocessed_data/word_ner_startindex_dict/dev_results.txt", "w") as out:
#         for sent in dev_sents: 
#             for i in range(len(sent)):
#                 word = sent[i][0]
#                 gold = sent[i][1]
#                 pred = y_dev_pred[j]
#                 j += 1
#                 out.write("{}\t{}\t{}\n".format(word,gold,pred))
#         out.write("\n")
        
    dev_pathRead = "/content/gdrive/Team Drives/cis530/raw_system_data/dev/system/"
    dev_pathOut = "/content/gdrive/Team Drives/cis530/raw_system_data/dev/system/"
#     #create an .ann format for predictions
    createAnnFormat(dev_dict, y_dev_pred,dev_pathRead, dev_pathOut)

    #Test Data
    test_feats = []
    test_labels = []

    # switch to test_sents for your final results
    for sent in test_sents:
        for i in range(len(sent)):
            feats = word2features(sent,i)
            test_feats.append(feats)
            test_labels.append(sent[i][1])

    X_test = vectorizer.transform(test_feats)
    y_test_pred = model.predict(X_test)

#     j = 0
#     print("Writing to test_results.txt")
#     # format is: word gold pred
#     with open("/content/gdrive/Team Drives/cis530/preprocessed_data/word_ner_startindex_dict/test_results.txt", "w") as out:
#         for sent in test_sents: 
#             for i in range(len(sent)):
#                 word = sent[i][0]
#                 gold = sent[i][1]
#                 pred = y_test_pred[j]
#                 j += 1
#                 out.write("{}\t{}\t{}\n".format(word,gold,pred))
#         out.write("\n")
        
    test_pathRead = "/content/gdrive/Team Drives/cis530/raw_system_data/test/system/"
    test_pathOut = "/content/gdrive/Team Drives/cis530/raw_system_data/test/system/"
#     #create an .ann format for predictions
    createAnnFormat(test_dict, y_test_pred,test_pathRead, test_pathOut)    

train_sents len=  8300
dev_sents len=  4048
test_sents len=  3231
Training the model


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Trained the model
Generating .ann for  S1139-76322016000300016-2
Generating .ann for  S1130-05582007000500007-1
Generating .ann for  S1130-01082008001000010-1
Generating .ann for  S0004-06142006000900008-1
Generating .ann for  S0004-06142009000400011-1
Generating .ann for  S1579-699X2004000400002-1
Generating .ann for  S0210-56912010000200009-1
Generating .ann for  S1130-01082008000900014-1
Generating .ann for  S0376-78922011000200004-1
Generating .ann for  S1139-76322014000100010-1
Generating .ann for  S0211-57352007000200017-1
Generating .ann for  S0376-78922009000400010-3
Generating .ann for  S1130-01082007000100009-1
Generating .ann for  S1130-14732009000300006-1
Generating .ann for  S0004-06142006000600009-1
Generating .ann for  S1139-76322016000400011-2
Generating .ann for  S0212-16112005000600011-1
Generating .ann for  S0004-06142005001000016-1
Generating .ann for  S0211-69952013000700025-1
Generating .ann for  S0211-69952017000200225-2
Generating .ann for  S0210-569120070007000

In [0]:
#create an .ann format for predictions
def createAnnFormat(preprocess_dict, y_pred, pathRead, pathOut):   
  
  
    j = 0
    for docId, v in preprocess_dict.items():
        print('Generating .ann for ',docId)
#         #Get the y_pred
#         X = preprocess_dict[docId]        
        #make predictions and store the results with their tags
        txt_start_end = ""
        tmp_tags=[]
        curr_tag = ""
        
        for sent in v:
          for item in sent:
            word = item[0]
            word_start= item[2]
            word_end = word_start + len(word) - 1
            pred = y_pred[j]   

            #if prediction is a beginning, add the tag to the tag list
            if pred[0:2] == "B-":
              if txt_start_end != "":
                tmp_tags.append((txt_start_end,curr_tag))
                txt_start_end= ""
              curr_tag= pred[2:]
              txt_start_end +=""+str(word_start)+"-"+str(word_end)+","
            #if it is a contuniation keep adding 
            elif pred[0:2] == "I-":
              txt_start_end +=""+str(word_start)+"-"+str(word_end)+","
            j += 1
        if txt_start_end != "":
          tmp_tags.append((txt_start_end,curr_tag))
          txt_start_end= ""  
        #we might be missing some chars in between values, so parse the data and 
        #get sentence matching that                        
        with open(pathRead+docId+".txt", "r") as f:                                                   
          complete_doc = f.read()       

        #now for all the text, get their exact match and create tags
        tags = []
        t_count = 1                        
        for i, i_tag in tmp_tags:
          splits = i.split(",")[:-1]
          beginning_split = splits[0].split("-")                 
          real_start = int(beginning_split[0])                      
          if len(splits) ==1:                  
            real_end = int(beginning_split[1])+1
          else:
            end_split = splits[len(splits)-1].split("-")
            real_end = int(end_split[1])+1
          tags.append("T"+str(t_count)+"\t"+i_tag+" "+str(real_start)+" "+str(real_end)+"\t"+ complete_doc[real_start:real_end])
          t_count += 1
        #now output all these
        with open(pathOut+docId+".ann", "w") as out:
          for i in tags:
            out.write(i+"\n")                      

In [0]:
#Evaluation
#!/usr/bin/env python

# Python version of the evaluation script from CoNLL'00-

# Intentional differences:
# - accept any space as delimiter by default
# - optional file argument (default STDIN)
# - option to set boundary (-b argument)
# - LaTeX output (-l argument) not supported
# - raw tags (-r argument) not supported

import sys
import re

from collections import defaultdict, namedtuple

ANY_SPACE = '<SPACE>'

class FormatError(Exception):
    pass

Metrics = namedtuple('Metrics', 'tp fp fn prec rec fscore')

class EvalCounts(object):
    def __init__(self):
        self.correct_chunk = 0    # number of correctly identified chunks
        self.correct_tags = 0     # number of correct chunk tags
        self.found_correct = 0    # number of chunks in corpus
        self.found_guessed = 0    # number of identified chunks
        self.token_counter = 0    # token counter (ignores sentence breaks)

        # counts by type
        self.t_correct_chunk = defaultdict(int)
        self.t_found_correct = defaultdict(int)
        self.t_found_guessed = defaultdict(int)

def parse_args(argv):
    import argparse
    parser = argparse.ArgumentParser(
        description='evaluate tagging results using CoNLL criteria',
        formatter_class=argparse.ArgumentDefaultsHelpFormatter
    )
    arg = parser.add_argument
    arg('-b', '--boundary', metavar='STR', default='-X-',
        help='sentence boundary')
    arg('-d', '--delimiter', metavar='CHAR', default=ANY_SPACE,
        help='character delimiting items in input')
    arg('-o', '--otag', metavar='CHAR', default='O',
        help='alternative outside tag')
    arg('file', nargs='?', default=None)
    return parser.parse_args(argv)

def parse_tag(t):
    m = re.match(r'^([^-]*)-(.*)$', t)
    return m.groups() if m else (t, '')

def evaluate(iterable, options=None):
    if options is None:
        options = parse_args([])    # use defaults

    counts = EvalCounts()
    num_features = None       # number of features per line
    in_correct = False        # currently processed chunks is correct until now
    last_correct = 'O'        # previous chunk tag in corpus
    last_correct_type = ''    # type of previously identified chunk tag
    last_guessed = 'O'        # previously identified chunk tag
    last_guessed_type = ''    # type of previous chunk tag in corpus

    for line in iterable:
        line = line.rstrip('\r\n')

        if options.delimiter == ANY_SPACE:
            features = line.split()
        else:
            features = line.split(options.delimiter)

        if num_features is None:
            num_features = len(features)
        elif num_features != len(features) and len(features) != 0:
            raise FormatError('unexpected number of features: %d (%d)' %
                              (len(features), num_features))

        if len(features) == 0 or features[0] == options.boundary:
            features = [options.boundary, 'O', 'O']
        if len(features) < 3:
            raise FormatError('unexpected number of features in line %s' % line)

        guessed, guessed_type = parse_tag(features.pop())
        correct, correct_type = parse_tag(features.pop())
        first_item = features.pop(0)

        if first_item == options.boundary:
            guessed = 'O'

        end_correct = end_of_chunk(last_correct, correct,
                                   last_correct_type, correct_type)
        end_guessed = end_of_chunk(last_guessed, guessed,
                                   last_guessed_type, guessed_type)
        start_correct = start_of_chunk(last_correct, correct,
                                       last_correct_type, correct_type)
        start_guessed = start_of_chunk(last_guessed, guessed,
                                       last_guessed_type, guessed_type)

        if in_correct:
            if (end_correct and end_guessed and
                last_guessed_type == last_correct_type):
                in_correct = False
                counts.correct_chunk += 1
                counts.t_correct_chunk[last_correct_type] += 1
            elif (end_correct != end_guessed or guessed_type != correct_type):
                in_correct = False

        if start_correct and start_guessed and guessed_type == correct_type:
            in_correct = True

        if start_correct:
            counts.found_correct += 1
            counts.t_found_correct[correct_type] += 1
        if start_guessed:
            counts.found_guessed += 1
            counts.t_found_guessed[guessed_type] += 1
        if first_item != options.boundary:
            if correct == guessed and guessed_type == correct_type:
                counts.correct_tags += 1
            counts.token_counter += 1

        last_guessed = guessed
        last_correct = correct
        last_guessed_type = guessed_type
        last_correct_type = correct_type

    if in_correct:
        counts.correct_chunk += 1
        counts.t_correct_chunk[last_correct_type] += 1

    return counts

def uniq(iterable):
  seen = set()
  return [i for i in iterable if not (i in seen or seen.add(i))]

def calculate_metrics(correct, guessed, total):
    tp, fp, fn = correct, guessed-correct, total-correct
    p = 0 if tp + fp == 0 else 1.*tp / (tp + fp)
    r = 0 if tp + fn == 0 else 1.*tp / (tp + fn)
    f = 0 if p + r == 0 else 2 * p * r / (p + r)
    return Metrics(tp, fp, fn, p, r, f)

def metrics(counts):
    c = counts
    overall = calculate_metrics(
        c.correct_chunk, c.found_guessed, c.found_correct
    )
    by_type = {}
    for t in uniq(list(c.t_found_correct) + list(c.t_found_guessed)):
        by_type[t] = calculate_metrics(
            c.t_correct_chunk[t], c.t_found_guessed[t], c.t_found_correct[t]
        )
    return overall, by_type

def report(counts, out=None):
    if out is None:
        out = sys.stdout

    overall, by_type = metrics(counts)

    c = counts
    out.write('processed %d tokens with %d phrases; ' %
              (c.token_counter, c.found_correct))
    out.write('found: %d phrases; correct: %d.\n' %
              (c.found_guessed, c.correct_chunk))

    if c.token_counter > 0:
        out.write('accuracy: %6.2f%%; ' %
                  (100.*c.correct_tags/c.token_counter))
        out.write('precision: %6.2f%%; ' % (100.*overall.prec))
        out.write('recall: %6.2f%%; ' % (100.*overall.rec))
        out.write('FB1: %6.2f\n' % (100.*overall.fscore))

    for i, m in sorted(by_type.items()):
        out.write('%17s: ' % i)
        out.write('precision: %6.2f%%; ' % (100.*m.prec))
        out.write('recall: %6.2f%%; ' % (100.*m.rec))
        out.write('FB1: %6.2f  %d\n' % (100.*m.fscore, c.t_found_guessed[i]))

def end_of_chunk(prev_tag, tag, prev_type, type_):
    # check if a chunk ended between the previous and current word
    # arguments: previous and current chunk tags, previous and current types
    chunk_end = False

    if prev_tag == 'E': chunk_end = True
    if prev_tag == 'S': chunk_end = True

    if prev_tag == 'B' and tag == 'B': chunk_end = True
    if prev_tag == 'B' and tag == 'S': chunk_end = True
    if prev_tag == 'B' and tag == 'O': chunk_end = True
    if prev_tag == 'I' and tag == 'B': chunk_end = True
    if prev_tag == 'I' and tag == 'S': chunk_end = True
    if prev_tag == 'I' and tag == 'O': chunk_end = True

    if prev_tag != 'O' and prev_tag != '.' and prev_type != type_:
        chunk_end = True

    # these chunks are assumed to have length 1
    if prev_tag == ']': chunk_end = True
    if prev_tag == '[': chunk_end = True

    return chunk_end

def start_of_chunk(prev_tag, tag, prev_type, type_):
    # check if a chunk started between the previous and current word
    # arguments: previous and current chunk tags, previous and current types
    chunk_start = False

    if tag == 'B': chunk_start = True
    if tag == 'S': chunk_start = True

    if prev_tag == 'E' and tag == 'E': chunk_start = True
    if prev_tag == 'E' and tag == 'I': chunk_start = True
    if prev_tag == 'S' and tag == 'E': chunk_start = True
    if prev_tag == 'S' and tag == 'I': chunk_start = True
    if prev_tag == 'O' and tag == 'E': chunk_start = True
    if prev_tag == 'O' and tag == 'I': chunk_start = True

    if tag != 'O' and tag != '.' and prev_type != type_:
        chunk_start = True

    # these chunks are assumed to have length 1
    if tag == '[': chunk_start = True
    if tag == ']': chunk_start = True

    return chunk_start



In [0]:
#Evaluate training results

with open('/content/gdrive/Team Drives/cis530/preprocessed_data/word_ner_startindex_dict/train_results.txt') as f:
    counts = evaluate(f)
report(counts)

processed 157882 tokens with 8616 phrases; found: 8968 phrases; correct: 8265.
accuracy:  99.47%; precision:  92.16%; recall:  95.93%; FB1:  94.01
            CALLE: precision:  65.08%; recall:  82.64%; FB1:  72.81  819
     CENTRO_SALUD: precision:  60.00%; recall:  75.00%; FB1:  66.67  5
CORREO_ELECTRONICO: precision:  88.00%; recall:  98.51%; FB1:  92.96  375
EDAD_SUJETO_ASISTENCIA: precision:  91.30%; recall:  96.04%; FB1:  93.61  851
FAMILIARES_SUJETO_ASISTENCIA: precision:  84.76%; recall:  80.35%; FB1:  82.49  164
           FECHAS: precision:  99.38%; recall:  98.18%; FB1:  98.78  975
         HOSPITAL: precision:  70.05%; recall:  83.33%; FB1:  76.12  207
 ID_ASEGURAMIENTO: precision:  98.62%; recall: 100.00%; FB1:  99.30  289
ID_CONTACTO_ASISTENCIAL: precision: 100.00%; recall:  98.08%; FB1:  99.03  51
ID_SUJETO_ASISTENCIA: precision:  98.57%; recall:  97.88%; FB1:  98.22  421
ID_TITULACION_PERSONAL_SANITARIO: precision:  98.88%; recall:  99.72%; FB1:  99.29  356
      INSTIT

In [0]:
#Evaluate dev results

with open('/content/gdrive/Team Drives/cis530/preprocessed_data/word_ner_startindex_dict/dev_results.txt') as f:
    counts = evaluate(f)
report(counts)


processed 74001 tokens with 4320 phrases; found: 4696 phrases; correct: 2943.
accuracy:  96.82%; precision:  62.67%; recall:  68.12%; FB1:  65.28
            CALLE: precision:  14.72%; recall:  27.44%; FB1:  19.16  591
     CENTRO_SALUD: precision:   0.00%; recall:   0.00%; FB1:   0.00  1
CORREO_ELECTRONICO: precision:  93.48%; recall:  96.63%; FB1:  95.03  184
EDAD_SUJETO_ASISTENCIA: precision:  85.89%; recall:  90.98%; FB1:  88.36  411
FAMILIARES_SUJETO_ASISTENCIA: precision:  70.49%; recall:  58.90%; FB1:  64.18  61
           FECHAS: precision:  54.90%; recall:  92.92%; FB1:  69.02  909
         HOSPITAL: precision:  37.60%; recall:  47.96%; FB1:  42.15  125
 ID_ASEGURAMIENTO: precision:  57.72%; recall:  64.66%; FB1:  60.99  149
ID_CONTACTO_ASISTENCIAL: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
ID_EMPLEO_PERSONAL_SANITARIO: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
ID_SUJETO_ASISTENCIA: precision:  61.11%; recall:   5.21%; FB1:   9.61  18
ID_TITULACION_PERSONAL

In [0]:

#Evaluate test results

with open('/content/gdrive/Team Drives/cis530/preprocessed_data/word_ner_startindex_dict/test_results.txt') as f:
    counts = evaluate(f)
report(counts)

processed 66194 tokens with 3516 phrases; found: 3887 phrases; correct: 2383.
accuracy:  96.88%; precision:  61.31%; recall:  67.78%; FB1:  64.38
            CALLE: precision:  15.43%; recall:  29.48%; FB1:  20.26  512
     CENTRO_SALUD: precision:  16.67%; recall:  33.33%; FB1:  22.22  6
CORREO_ELECTRONICO: precision:  88.08%; recall:  98.52%; FB1:  93.01  151
EDAD_SUJETO_ASISTENCIA: precision:  83.38%; recall:  91.08%; FB1:  87.06  355
FAMILIARES_SUJETO_ASISTENCIA: precision:  70.83%; recall:  59.30%; FB1:  64.56  72
           FECHAS: precision:  52.64%; recall:  98.19%; FB1:  68.54  720
         HOSPITAL: precision:  44.04%; recall:  53.33%; FB1:  48.24  109
 ID_ASEGURAMIENTO: precision:  46.75%; recall:  56.25%; FB1:  51.06  154
ID_CONTACTO_ASISTENCIAL: precision:   0.00%; recall:   0.00%; FB1:   0.00  1
ID_SUJETO_ASISTENCIA: precision:  50.00%; recall:   0.61%; FB1:   1.21  2
ID_TITULACION_PERSONAL_SANITARIO: precision:  73.14%; recall:  93.43%; FB1:  82.05  175
      INSTITUCION